# Лабораторная работа No 9
## Алгоритмы кластерного анализа данных

In [18]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_rows', None)

Исследуем полученные данные

In [2]:
df = pd.read_csv('colleges.csv')
df = df[['okrug', 'X', 'Y']]
df

,okrug,X,Y
0,Юго-Восточный административный округ,"37,850588","55,685852"
1,Южный,"37,657122","55,713902"
2,Центральный административный округ,"37,625878","55,73442"
3,Южный административный округ,"37,654086","55,726615"
4,Южный административный округ,"37,695225","55,688223"
...,...,...,...
280,Центральный административный округ,"37,68165","55,74362"
281,Центральный административный округ,"37,684059","55,736391"
282,Центральный административный округ,"37,684201","55,736755"
283,Восточный административный округ,"37,816595","55,789968"


Координаты не заданы при помощи float-числа. Приведем типы:

In [3]:
df[['X', 'Y']] = df[['X', 'Y']].apply(lambda x: x.str.replace(',', '.', 1)).apply(pd.to_numeric)
df

,okrug,X,Y
0,Юго-Восточный административный округ,37.850588,55.685852
1,Южный,37.657122,55.713902
2,Центральный административный округ,37.625878,55.734420
3,Южный административный округ,37.654086,55.726615
4,Южный административный округ,37.695225,55.688223
...,...,...,...
280,Центральный административный округ,37.681650,55.743620
281,Центральный административный округ,37.684059,55.736391
282,Центральный административный округ,37.684201,55.736755
283,Восточный административный округ,37.816595,55.789968


Найдем округа, зайдествованные в выборке

In [4]:
set(df.okrug)

{'Восточный административный округ',
 'Западный административный округ',
 'Зеленоградский административный округ',
 'Северный административный округ',
 'Северо-Восточный административный округ',
 'Северо-Западный административный округ',
 'Центральный административный округ',
 'Юго-Восточный административный округ',
 'Юго-Западный административный округ',
 'Южный',
 'Южный административный округ'}

У некоторых записей есть ошибка в метке округа - исправим.

In [5]:
df.loc[df.okrug == 'Южный', 'okrug'] = 'Южный административный округ'

In [6]:
set(df.okrug)

{'Восточный административный округ',
 'Западный административный округ',
 'Зеленоградский административный округ',
 'Северный административный округ',
 'Северо-Восточный административный округ',
 'Северо-Западный административный округ',
 'Центральный административный округ',
 'Юго-Восточный административный округ',
 'Юго-Западный административный округ',
 'Южный административный округ'}

Ошибка исправлена

Найдем и подготовим данные центров округов

In [7]:
# Данные по округам с википедии и гугл-карт (скорее всего являются центром массы, но не фактическим центром)
data = [
        # ['Восточный административный округ', 'ВАО', 55.7752, 37.9056],
        ['Западный административный округ', 'ЗАО', 55.6756, 37.2327],
        ['Зеленоградский административный округ', 'ЗелАО', 55.9791, 37.2001],
        ['Северный административный округ', 'САО', 55.8587345, 37.3223247],
        ['Северо-Восточный административный округ', 'СВАО', 55.8562, 37.6138],
        ['Северо-Западный административный округ', 'СЗАО', 55.8131, 37.4201],
        ['Центральный административный округ', 'ЦАО', 55.744, 37.6165],
        ['Юго-Восточный административный округ', 'ЮВАО', 55.6828, 37.8053],
        ['Юго-Западный административный округ', 'ЮЗАО', 55.5756, 37.5444],
        ['Южный административный округ', 'ЮАО', 55.6328, 37.6749]]

reference = pd.DataFrame(data, columns=['full', 'short', 'X', 'Y'])
reference

,full,short,X,Y
0,Восточный административный округ,ВАО,55.775200,37.905600
1,Западный административный округ,ЗАО,55.675600,37.232700
2,Зеленоградский административный округ,ЗелАО,55.979100,37.200100
3,Северный административный округ,САО,55.858734,37.322325
4,Северо-Восточный административный округ,СВАО,55.856200,37.613800
5,Северо-Западный административный округ,СЗАО,55.813100,37.420100
6,Центральный административный округ,ЦАО,55.744000,37.616500
7,Юго-Восточный административный округ,ЮВАО,55.682800,37.805300
8,Юго-Западный административный округ,ЮЗАО,55.575600,37.544400
9,Южный административный округ,ЮАО,55.632800,37.674900


In [20]:
Xs = [(row['X'], row['Y']) for idx, row in df.iterrows()]
Ys = [(row['X'], row['Y']) for idx, row in reference.iterrows()]

In [25]:
def distance(X, Y):
    x1, y1 = X
    x2, y2 = Y
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

def clusterize(Xs, Ys):
    clusters = [list() for _ in Ys]
    for idx, X in enumerate(Xs):
        distances = [distance(X, Y) for Y in Ys]
        min_idx = np.argmin(distances)
        print(distances)
        clusters[min_idx].append((idx, X))
    return clusters

In [26]:
clusterize(Xs, Ys)

.855659450657107, 25.667184622723116, 25.489976367037965, 25.60077332031894, 25.547534860572245]
[25.581684054176666, 25.987706656038675, 26.223825704910354, 26.052284468303608, 25.844316468332394, 25.950747186343534, 25.762907734910605, 25.58634452995419, 25.69527407596418, 25.64298717704934]
[25.423713942736217, 25.831696818847966, 26.067119178913504, 25.895576968976897, 25.686881378175904, 25.79390960330907, 25.605749292377286, 25.42886176463429, 25.53873133523433, 25.48596404407885]
[25.483695187231113, 25.889153515893607, 26.125464876202802, 25.95392430982964, 25.74616677203242, 25.852424153601266, 25.664677345992608, 25.488209536260214, 25.59686843131609, 25.544717985468274]
[25.48336840094363, 25.888835541449374, 26.125143806521187, 25.953603228178395, 25.745842414801896, 25.8521024851638, 25.664354229744653, 25.48788495239674, 25.596548069843564, 25.544395471245984]
[25.48227220604607, 25.887869916024925, 26.12413249864248, 25.952591748365464, 25.74478237763583, 25.851082325941

[[(0, (37.850588, 55.685852000000004)),
  (1, (37.657121999999994, 55.713902000000004)),
  (2, (37.625878, 55.73441999999999)),
  (3, (37.654086, 55.726615)),
  (4, (37.695225, 55.688223)),
  (5, (37.609058000000005, 55.714889)),
  (6, (37.432015, 55.829499)),
  (7, (37.394847, 55.793141000000006)),
  (8, (37.628239, 55.803895)),
  (9, (37.796336, 55.809568000000006)),
  (10, (37.625209000000005, 55.734007)),
  (11, (37.572617, 55.670439)),
  (12, (37.566239, 55.76425)),
  (13, (37.531534, 55.737669)),
  (14, (37.636379999999996, 55.75948199999999)),
  (15, (37.790465999999995, 55.797734999999996)),
  (16, (37.585937, 55.750604)),
  (17, (37.551662, 55.844602)),
  (18, (37.659394, 55.88466999999999)),
  (19, (37.661277, 55.593499)),
  (20, (37.616458, 55.806403)),
  (21, (37.349534999999996, 55.850168999999994)),
  (22, (37.770481, 55.713573)),
  (23, (37.702604, 55.810354000000004)),
  (24, (37.51632, 55.823067)),
  (25, (37.440805, 55.84399200000001)),
  (26, (37.708019, 55.619849)),